In [5]:
import numpy as np
import dash
from dash import dcc
from dash import html
import plotly.express as px
import plotly.graph_objects as go
from dash.dependencies import Input, Output
import pandas as pd


In [6]:
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
df.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [7]:
df['Launch Site'].unique()

array(['CCAFS LC-40', 'VAFB SLC-4E', 'KSC LC-39A', 'CCAFS SLC-40'],
      dtype=object)

In [109]:
min_value = df['Payload Mass (kg)'].min()
max_value = df['Payload Mass (kg)'].max()

app = dash.Dash(__name__)
app.layout = html.Div(children= [
    html.H1('Space X Dashboard', style= {'textAlign': 'center', 'font-size': 42}),
    
    html.Div([
        html.H3('Select the Launch Site', style={'textAlign': 'center', 'font-size': 14}),
        dcc.Dropdown(id = 'site-dropdown',
                    options = [ 
                    {'label': 'All Sites', 'value': 'ALL'},
                    {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                    {'label': 'VAFB SLC-4E', 'value':'VAFB SLC-4E'},
                    {'label': 'KSC LC-39A', 'value':'KSC LC-39A'},
                    {'label':'CCAFS SLC-40', 'value':'CCAFS SLC-40'}],
                    value = 'ALL',
                    placeholder= 'Select the Launch Site',
                    searchable= True),
        html.Div(dcc.Graph('success-pie-chart')),
        html.Br(),
        html.Br(),
        html.Div([html.H3('Select Payload mass (Kg)', style = {'textAlign': 'center', 'font-size':13}),
                  dcc.RangeSlider(id= 'payload-slider',
                                min = 0, max = 10000, step = 500,
                                 marks = {x:'{0}'.format(x) for x in range(0,10001,500)},
                                 value = [min_value, max_value])]),
        html.Br(),
        html.Br(),
        html.Div([html.P('Scatter Plot', style={'textAlign': 'center', 'font-size':14}),
                 html.Br(),
                 dcc.Graph('success-payload-scatter-chart')])
        
    ])
    
])


@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    Input(component_id= 'site-dropdown', component_property= 'value')
    
)

def pie_chart(entered_site):
    
    if entered_site == 'ALL':
        fig = px.pie(values = df.groupby('Launch Site')['class'].mean(),
                     names = df.groupby('Launch Site')['Launch Site'].first(),
                     title = 'Total Launches for site {}'.format(entered_site))
        
    else:
        
        fig = px.pie(values = df[df['Launch Site'] == str(entered_site)]['class'].value_counts(normalize = True),
            names = df['class'].unique(),
            title = 'Total Launches for site {}'.format('entered_site'))
        
        
    
    return fig
    
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'), 
     Input(component_id="payload-slider", component_property="value")]
)

def scatter_chart(entered_site, payload_range):
    if entered_site == 'ALL':
        fig = px.scatter(df[df['Payload Mass (kg)'].between(int(payload_range[0]), int(payload_range[1]))],
                         x = 'Payload Mass (kg)',
                         y = 'class',
                         color= 'Booster Version Category',
                         title = 'Correlation between Payload Mass and Success for site {}between from mass{:6,d}Kg to{:7,d}Kg.'.format(entered_site,payload_range[0], payload_range[1]))
    
    else:
        filtered_df = df[df['Launch Site'] == str(entered_site)]
        fig = px.scatter(filtered_df[filtered_df['Payload Mass (kg)'].between(payload_range[0],payload_range[1])], 
                         x = 'Payload Mass (kg)', 
                         y = 'class', 
                         color = 'Booster Version Category',
                         title = 'Correlation between Payload Mass and Success for site {}between from mass{:6,d}Kg to{:7,d}Kg.'.format(entered_site,payload_range[0], payload_range[1]))
                        
    
    return fig



if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [11/Apr/2023 02:46:40] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2023 02:46:41] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2023 02:46:41] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2023 02:46:41] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2023 02:46:41] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2023 02:46:41] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2023 02:46:41] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2023 02:46:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2023 02:46:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2023 02:47:22] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2023 02:47:22] "POST /_dash-